In [2]:
import pandas as pd


In [3]:
crime_df = pd.read_stata('/Users/salma/Research/dissertation_work/data/crime_jkap/ucr_offenses_known_yearly_1960_2020_dta/offenses_known_yearly_1960_2020.dta')
print(list(crime_df))
## Get the data after 2010 for agencies with pop > 10,000 and reported data for all months
crime_df_10_20  = crime_df.query('year > 2009 and population > 10000 and number_of_months_missing == 0')
print(crime_df_10_20['ori'].nunique())

['ori', 'ori9', 'agency_name', 'state', 'state_abb', 'year', 'number_of_months_missing', 'last_month_reported', 'arson_number_of_months_missing', 'arson_last_month_reported', 'fips_state_code', 'fips_county_code', 'fips_state_county_code', 'fips_place_code', 'agency_type', 'crosswalk_agency_name', 'census_name', 'longitude', 'latitude', 'address_name', 'address_street_line_1', 'address_street_line_2', 'address_city', 'address_state', 'address_zip_code', 'population_group', 'population_1', 'population_1_county', 'population_2', 'population_2_county', 'population_3', 'population_3_county', 'population', 'country_division', 'juvenile_age', 'core_city_indication', 'fbi_field_office', 'followup_indication', 'zip_code', 'month_included_in', 'covered_by_ori', 'agency_count', 'special_mailing_group', 'special_mailing_address', 'first_line_of_mailing_address', 'second_line_of_mailing_address', 'third_line_of_mailing_address', 'fourth_line_of_mailing_address', 'officers_killed_by_felony', 'offic

In [5]:
## Get the required columns ##
crime_df_10_20_req = crime_df_10_20.loc[:, ['ori', 'agency_name', 'state', 'year', 'fips_state_code', 'longitude', 'latitude',
                                            'fips_county_code', 'fips_place_code', 'population', 'actual_murder', 
                                            'actual_manslaughter', 'actual_rape_total', 'actual_robbery_total',
                                            'actual_assault_simple', 'actual_burg_total', 'actual_theft_total',
                                            'actual_mtr_veh_theft_total', 
                                            'actual_assault_aggravated', 'actual_arson_grand_total']]

## Create new columns ##
crime_df_10_20_req['murder'] = crime_df_10_20_req['actual_murder'] + crime_df_10_20_req['actual_manslaughter']

crime_df_10_20_req.drop(['actual_murder', 'actual_manslaughter'], axis=1, inplace=True)

crime_df_10_20_req.rename({'ori': 'ORI','fips_state_code': 'state_fips',
                          'fips_county_code': 'county_fips', 'fips_place_code': 'place_fips', 
                          'actual_rape_total': 'rape', 'actual_robbery_total': 'robbery', 
                          'actual_assault_simple': 'simple_assault', 'actual_burg_total': 'burglary',
                          'actual_theft_total': 'larceny',
                          'actual_mtr_veh_theft_total': 'motor_vehicle_theft', 'actual_assault_aggravated': 'agg_assault',
                          'actual_arson_grand_total': 'arson'
                          }, axis=1, inplace=True)

num = crime_df_10_20_req._get_numeric_data()

num[num < 0] = 0

crime_df_10_20_req.fillna(0, inplace=True)

print(crime_df_10_20_req['ORI'].nunique())

print(crime_df_10_20_req.shape[0])

5810
58897


In [95]:
crime_df_10_20_req.to_csv('/Users/salma/Research/dissertation_work/data/crime_jkap/crime_10_20_req.csv', index=False)

### Getting the ORIs which have data for all years from 2016 - 2020 ##

In [6]:
years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
all_rptd_oris = []

for ori in crime_df_10_20_req['ORI'].unique():
    ori_years = crime_df_10_20_req.query(f'ORI == "{ori}"')['year'].tolist()
    ori_years.sort()
    
    if ori_years == years:
        all_rptd_oris.append(ori)
        
## Now get only those records for ORIs which are in all_rptd_oris ##
crime_df_10_20_req_all_rept_oris = crime_df_10_20_req.loc[crime_df_10_20_req['ORI'].isin(all_rptd_oris)]

print(crime_df_10_20_req_all_rept_oris['ORI'].nunique())
print(crime_df_10_20_req_all_rept_oris.shape[0])

crime_df_10_20_req_all_rept_oris.to_csv('/Users/salma/Research/dissertation_work/data/crime_jkap/crime_10_20_req_all_rept_oris.csv',
                                       index = False)

4448
48928


### Does each ORI have multiple fips under it? Multiple states under each ORI unlikely but may be multiple counties or places?

In [ ]:
### Let's check with the crime_df_10_20_req_all_rept_oris df
print(list(crime_df_10_20_req_all_rept_oris))

In [ ]:
test_df_1ori = crime_df_10_20_req_all_rept_oris.query('ORI == "AK00101"')[['AGENCY', 'state', 'YEAR', 'STATEFP', 'CNTY', 'place_fips', 'longitude', 'latitude', 'population']]
test_df_1ori

### Each ORI has corresponding FIPS – so assigning the corresponding all values to that ORI